# Vietnamese Currency Recognition (Colab)

Notebook này sẽ:
- Cài dependencies
- Tải dataset từ KaggleHub (`nguyentrongdai/vietnamese-currency`)
- Chuẩn hoá dataset về dạng `dataset/<class_name>/*.png`
- Huấn luyện model (ResNet18) và lưu `artifacts/best_model.pt`
- Demo suy luận với ảnh mẫu hoặc upload ảnh


In [ ]:
# Cài thư viện cần thiết (Colab thường đã có sẵn torch/torchvision)
!pip -q install -U kagglehub numpy opencv-python pillow

import os
from pathlib import Path
print('Working dir:', os.getcwd())
print('Files:', [p.name for p in Path(".").iterdir()][:20])

## Code trên Colab
Notebook sẽ tự kiểm tra nếu chưa có `train.py` thì sẽ **clone repo** từ GitHub vào Colab và `cd` vào repo.

Nếu bạn đã mở notebook trực tiếp từ repo (hoặc đã upload đầy đủ file code), cell clone sẽ không clone lại.

In [ ]:
# Đảm bảo đang có source code (train.py, predict.py, ...)
# Nếu notebook chạy trong Colab runtime mới, thường thư mục hiện tại không có repo.

from pathlib import Path
import os

if not Path('train.py').exists():
    # Clone repo về Colab
    !git clone https://github.com/conghungtran/Money-detection.git
    os.chdir('Money-detection')

print('cwd:', os.getcwd())
print('has train.py:', Path('train.py').exists())
print('files:', [p.name for p in Path('.').iterdir()][:30])

## Tải dataset bằng KaggleHub

Nếu bạn gặp lỗi kiểu `401/403` khi download, bạn cần Kaggle API token:
- Vào Kaggle > Account > Create New Token để tải `kaggle.json`
- Upload `kaggle.json` lên Colab và đặt vào `/root/.kaggle/kaggle.json`

Sau đó chạy lại cell download.

In [ ]:
import os
from pathlib import Path

# KaggleHub có thể cần Kaggle credentials trong một số môi trường.
# Nếu cell download bị lỗi 401/403, hãy upload kaggle.json (Kaggle API token)
# vào /root/.kaggle/kaggle.json rồi chạy lại.

import kagglehub

path = kagglehub.dataset_download("nguyentrongdai/vietnamese-currency")
print("Path to dataset files:", path)
print("Exists:", Path(path).exists())

In [ ]:
import shutil
from pathlib import Path

# Chuẩn hoá dataset về ./dataset/<class>/* để train.py dùng torchvision.datasets.ImageFolder

def find_imagefolder_root(download_path: str) -> Path:
    root = Path(download_path)

    # Duyệt xuống vài mức, chọn thư mục có nhiều subfolder 6 chữ số nhất
    candidates = [root]
    for _ in range(4):
        new = []
        for c in candidates:
            if c.is_dir():
                for p in c.iterdir():
                    if p.is_dir():
                        new.append(p)
        candidates += new

    best = None
    best_score = -1
    for c in candidates:
        if not c.is_dir():
            continue
        subs = [p for p in c.iterdir() if p.is_dir()]
        score = sum(1 for p in subs if p.name.isdigit() and len(p.name) == 6)
        if score > best_score:
            best_score = score
            best = c

    if best is None or best_score <= 0:
        raise RuntimeError("Không tìm thấy thư mục dạng ImageFolder trong dataset download.")

    return best

src_root = find_imagefolder_root(path)
print('Detected ImageFolder root:', src_root)

dst_root = Path('dataset')
dst_root.mkdir(parents=True, exist_ok=True)

# Nếu đã có dataset rồi thì không copy lại
existing = [p for p in dst_root.iterdir() if p.is_dir()]
if existing:
    print('dataset/ already exists, skip copying. Example:', existing[0])
else:
    class_dirs = [p for p in src_root.iterdir() if p.is_dir() and p.name.isdigit() and len(p.name) == 6]
    print('Found classes:', sorted([p.name for p in class_dirs]))

    for cd in class_dirs:
        target = dst_root / cd.name
        target.mkdir(parents=True, exist_ok=True)
        for ext in ('*.png', '*.jpg', '*.jpeg', '*.PNG', '*.JPG', '*.JPEG'):
            for fp in cd.glob(ext):
                out = target / fp.name
                if not out.exists():
                    shutil.copy2(fp, out)

# Thống kê
counts = {}
for cd in sorted(dst_root.iterdir()):
    if cd.is_dir():
        n = len(list(cd.glob('*.png'))) + len(list(cd.glob('*.jpg'))) + len(list(cd.glob('*.jpeg')))
        counts[cd.name] = n
counts

## Huấn luyện
Lưu model vào `artifacts/best_model.pt`


In [ ]:
# Train (có thể bật GPU trong Runtime > Change runtime type)
# Giảm epochs nếu bạn muốn chạy nhanh để test.

!python3 train.py --data-dir dataset --output-dir artifacts --epochs 12 --batch-size 32

## Demo predict với ảnh mẫu trong dataset


In [ ]:
from pathlib import Path

# Chọn ảnh mẫu bất kỳ từ một class (ưu tiên 000200 nếu có)
preferred = Path('dataset/000200')
if preferred.exists():
    sample = next(preferred.glob('*.png'))
else:
    # fallback: tìm bất kỳ ảnh nào
    sample = next(Path('dataset').rglob('*.png'))

print('sample:', sample)
!python3 predict.py --model artifacts/best_model.pt --image {str(sample)} --topk 3

## Upload ảnh để nhận diện
Chạy cell dưới và chọn ảnh từ máy của bạn.


In [ ]:
# Upload ảnh từ máy của bạn để nhận diện
from google.colab import files

uploaded = files.upload()
paths = list(uploaded.keys())
print('uploaded:', paths)

if paths:
    img_path = paths[0]
    !python3 predict.py --model artifacts/best_model.pt --image {img_path} --topk 3

## (Tuỳ chọn) Chạy web demo (Streamlit)
Colab có thể chạy Streamlit nhưng cần tunnel/port-forward. Nếu bạn muốn mình cấu hình phần này, nói mình biết (mình sẽ thêm cell dùng `pyngrok`).
